In [ ]:
----------------------------
ส่วนของคำสั่ง import
----------------------------

# ฟังก์ชันเชื่อมต่อฐานข้อมูล
def con_db():
    global cs, con
    con = pymysql.connect(host="localhost", user="root", passwd="1234", 
                          db="register", charset="utf8")
    cs = con.cursor()

# สร้างฟังก์ชัน teacher สำหรับจัดการข้อมูลอาจารย์
def teacher():
    mainFrm.withdraw()  # เมื่อหน้าจอ teacher ขึ้นมา หน้าจอหลักจะถูกปิด

    # ฟังก์ชันปิดหน้าต่างเฟรม teacher 
    def showMainFrm():
        frmTeacher.destroy()  # ปิดหน้าจอ teacher
        mainFrm.update()       
        mainFrm.deiconify()    # แสดงหน้าจอหลัก

    # ฟังก์ชันลบข้อมูลออกจาก Entry widget
    def clearEntryOfTeacher():
        entID.delete(0, END)
        entFname.delete(0, END)
        entLname.delete(0, END)
        entEmail.delete(0, END)
        entTel.delete(0, END)

    # ฟังก์ชันแสดงข้อมูลที่ Entry widget เมื่อคลิกที่แถว Treeview widget 
    def SelectTreeTeacher(event):
        clearEntryOfTeacher() # เรียกใช้ฟังก์ชันเพื่อล้างข้อมูลออกจาก Entry widget
        # กำหนดให้แสดงข้อมูลที่ Entry widget เมื่อคลิกที่แถวของ Treeview widget
        item = viewTcr.selection()
        for i in item:
            entID.insert("", viewTcr.item(i, "values")[0])
            entFname.insert("", viewTcr.item(i, "values")[1])
            entLname.insert("", viewTcr.item(i, "values")[2])
            entEmail.insert("", viewTcr.item(i, "values")[3])
            entTel.insert("", viewTcr.item(i, "values")[4])

    # ฟังก์ชันให้แสดงผลข้อมูลจากตาราง teachers
    def insertDataToTreeView():
        # ลบข้อมูลออกจาก Treeview widgit
        for c in viewTcr.get_children():
            viewTcr.delete(c)

        con_db()  # เรียกใช้งานฟังก์ชันเชื่อมต่อฐานข้อมูล
        cs.execute("select * from teachers") # ใช้คำสั่ง select ดึงข้อมูลจากตาราง teacher
        data = cs.fetchall() # แสดงผลข้อมูลทั้งหมดในตาราง teacher
        # อ่านข้อมูลและเพิ่มข้อมูลไปยัง Treeview widget ทีละแถว
        for d in data:
            viewTcr.insert("", "end", values=d)
        con.close()
        cs.close()

    # ฟังก์ชันเพิ่มข้อมูลเข้าตาราง Teachers
    def insert_data():
        # ตรวจสอบ Entry widget มีค่าว่างหรือไม่
        if entID.get()=="" or entFname.get() =="" or entLname.get() == "" or 
            entEmail.get() == "" or entTel.get() =="":
            # แจ้งเตือนข้อความ
            messagebox.showwarning("Warning", "กรุณากรอกข้อมูลให้ครบด้วยครับ !!")
        else:
            con_db() # เรียกใช้ฟังก์ชันเชื่อมต่อฐานข้อมูล
            # คำสั่งเพิ่มข้อมูลลงตาราง teacher
            cs.execute("INSERT INTO teachers(teacher_id, f_name, l_name, e_mail, tel)"\
                       "VALUES(%s, %s, %s, %s, %s)"\
                       ,(str(entID.get()), str(entFname.get()),  
                         str(entLname.get()), str(entEmail.get()), str(entTel.get())))
            con.commit()    # คำสั่งปรับปรุงข้อมูลในฐานข้อมูล
            con.close()
            cs.close()
            clearEntryOfTeacher()   # เรียกใช้ฟังก์ชันเพื่อล้างข้อมูลออกจาก Entry widget
            insertDataToTreeView()  # เรียกใช้ฟังก์ชันเพื่อปรับปรุงข้อมูลใน Treeview widget
            # แสดง messageBox
            messagebox.showwarning("Information", "บันทึกข้อมูลเรียบร้อยแล้ว ครับ")  

    # ฟังก์ชันลบข้อมูลออกจากตาราง teachers
    def delete_data():
        if not viewTcr.selection():
            messagebox.showinfo("Information", "คุณยังไม่เลือกข้อมูลที่ต้องการลบ")
        else:
            item = viewTcr.selection()
            for i in item:
                con_db()      # เรียกใช้งานเชื่อมต่อฐานข้อมูล
                # คำสั่งลบข้อมูลออกจากตาราง teachers
                cs.execute("DELETE FROM teachers WHERE teacher_id = %s " % 
                           viewTcr.item(i, "values")[0])
                # ถ้าตอบ Yes เป็นการยืนยันการลบข้อมูล
                msgBox = messagebox.askquestion("Warning", "ต้องการลบข้อมูลนี้ใช่หรือไม่", 
                                                icon="warning")
                if msgBox == "yes":
                    con.commit()  # ยืนยันการลบข้อมูล
            con.close()
            cs.close()
            clearEntryOfTeacher()  # ลบข้อมูลใน Entry widget
            insertDataToTreeView() # ปรับปรุงข้อมูลใน Treeview widget

    # ฟังก์ชันปรับปรุงข้อมูลตาราง teachers
    def udate_data():
        # กำหนดให้ Entry widget ของ entID แสดงผลข้อมูลได้อย่างเดียว
        entStyle.configure("entID.TEntry", state="readonly")   
        # แจ้งเตือนถ้ายังไม่มีการเลือกข้อมูลที่ต้องการแก้ไข
        if not viewTcr.selection():
            messagebox.showinfo("Warning", "คุณยังไม่เลือกข้อมูลที่ต้องการแก้ไข")
        else:
            con_db()   # เรียกใช้ฟังก์ชันเชื่อมต่อกับฐานข้อมูล
            cs.execute("UPDATE teachers set teacher_id=%s, f_name=%s, l_name=%s,"\
                       "e_mail=%s, tel=%s WHERE teacher_id =%s "\
            ,(str(entID.get()), str(entFname.get()), str(entLname.get()), 
              str(entEmail.get()), str(entTel.get()), str(entID.get())))
        # แจ้งเตือนก่อนปรับปรุงข้อมูลด้วย messagebox ถ้าตอบ yes ข้อมูลจะถูกปรับปรุง
            msgBox = messagebox.askquestion("Information", "ต้องการแก้ไขข้อมูลนี้ใช่หรือไม่", 
                                            icon="warning")
            if msgBox == "yes":
                con.commit()
            con.close()
            cs.close()
            clearEntryOfTeacher()  # เรียกใช้ฟังก์ชันลบข้อมูลออกจาก Entry widget
            insertDataToTreeView() # เรียกใช้ฟังก์ชันเพื่อปรับปรุงข้อมูล Treeview widget

    # คำสั่งสร้าง Treeview widget
    frmTeacher = Toplevel()    # กำหนดให้ปรากฏขึ้นมาเป็นหน้าต่างบนสุด
    frmTeacher.title("อาจารย์ผู้สอน") # แสดงข้อความที่ title bar
    ttk.Frame(frmTeacher, height=410, width=750).pack() # กำหนดขนาด Treeview widget
    frmTeacher.resizable(width=0, height=0) # กำหนดไม่ให้สามารถปรับขนาดได้

    # กำหนดขนาดและจำนวนคอลัมน์ Treeview widget
    viewTcr = ttk.Treeview(frmTeacher, column=("ID", "Fname", "Lname", 
                                               "Email", "Tell"), show="headings")
    viewTcr.place(height=200, width=720, x=10, y=200)
    
    # กำหนดให้แสดง scrollbar 
    sc = ttk.Scrollbar(frmTeacher, orient="vertical", 
                       command=viewTcr.yview)
    sc.place(height=200, x=720, y=200)  # กำหนดความยาวและตำแหน่ง scrollbar
    viewTcr.configure(yscrollcommand=sc.set)

    # กำหนดขนาดความกว้างของคอลัมน์ Treeview widget
    viewTcr.column("#1", width="80")
    viewTcr.column("#2", width="150")
    viewTcr.column("#3", width="150")
    viewTcr.column("#4", width="200")
    viewTcr.column("#5", width="100")

    # กำหนดที่หัวแต่ละคอลัมน์ 
    viewTcr.heading("#1", text="รหัสอาจารย์")
    viewTcr.heading("#2", text="ชื่อ")
    viewTcr.heading("#3", text="นามสกุล")
    viewTcr.heading("#4", text="อีเมล์")
    viewTcr.heading("#5", text="เบอร์โทรศัพท์")
    # กำหนดให้เมื่อคลิกที่ Treeview widget ข้อมูลแต่ละคอลัมน์จะแสดงผลที่ 
    # Entry widget ผ่านฟังก์ชัน SelectTreeTeacher
    viewTcr.bind('<ButtonRelease>', SelectTreeTeacher)  
                 
    # เรียกใช้ฟังก์ชันแสดงข้อมูลใน TreeView widget ทันทีเมื่อแสดงผลหน้าต่างอาจารย์
    insertDataToTreeView() 

    # สร้างเฟรม label widget
    ttk.LabelFrame(frmTeacher, text="เพิ่มหรือแก้ไขข้อมูลอาจารย์").place(height=185, width=400, 
                                                                   x=10, y=10)

    # สร้าง label widget และกำหนดตำแหน่ง
    ttk.Label(frmTeacher, text="รหัสอาจารย์ :").place(height=20, width=60, 
                                                    x=20, y=40)
    ttk.Label(frmTeacher, text="ชื่อ :").place(height=20, width=60, 
                                             x=20, y=70)
    ttk.Label(frmTeacher, text="นามสกุล :").place(height=20, width=60, 
                                             x=20, y=100)
    ttk.Label(frmTeacher, text="อีเมล์ :").place(height=20, width=60, 
                                             x=20, y=130)
    ttk.Label(frmTeacher, text="เบอร์โทร :").place(height=20, width=60, 
                                             x=20, y=160)

    # สร้าง Entry widget 
    entStyle = ttk.Style()
    entID = ttk.Entry(frmTeacher, style="entID.TEntry")
    entFname = ttk.Entry(frmTeacher)
    entLname = ttk.Entry(frmTeacher)
    entEmail = ttk.Entry(frmTeacher)
    entTel = ttk.Entry(frmTeacher)
    
    # กำหนดตำแหน่งและขนาดของ entry widget
    entID.place(height=25, width=100, x=90, y=40)
    entFname.place(height=25, width=200, x=90, y=70)
    entLname.place(height=25, width=200, x=90, y=100)
    entEmail.place(height=25, width=200, x=90, y=130)
    entTel.place(height=25, width=100, x=90, y=160)

    # กำหนดรูปแบบ button widget
    btnStyle = ttk.Style()
    btnStyle.configure("btnSave.TButton", font=("Times", "10", "bold"))
    btnStyle.configure("btnEdit.TButton", font=("Times", "10", "bold"))
    btnStyle.configure("btnDelete.TButton", font=("Times", "10", "bold"))
    btnStyle.configure("btnClear.TButton", font=("Times", "10", "bold"))
    btnStyle.configure("btnExit.TButton", font=("Times", "10", "bold"))

    # สร้าง Button widget
    btnSave = ttk.Button(frmTeacher, text="บันทึก", command=insert_data, 
                         style="btnSave.TButton")
    btnEdit = ttk.Button(frmTeacher, text="แก้ไข", command=udate_data, 
                         style="btnEdit.TButton")
    btnDelete = ttk.Button(frmTeacher, text="ลบ", command=delete_data, 
                           style="btnDelete.TButton")
    btnCancle = ttk.Button(frmTeacher, text="ยกเลิก", command=clearEntryOfTeacher, 
                           style="btnClear.TButton")
    btnClose = ttk.Button(frmTeacher, text="ปิดโปรแกรม", command=showMainFrm, 
                          style="btnExit.TButton")

    # กำหนดตำแหน่งและขนาดของ Button widget
    btnSave.place(height=35, width=100, x=420, y=20)
    btnEdit.place(height=35, width=100, x=420, y=55)
    btnDelete.place(height=35, width=100, x=420, y=90)
    btnCancle.place(height=35, width=100, x=420, y=125)
    btnClose.place(height=35, width=100, x=420, y=160)
    
------------------------------------------
ส่วนของคำสั่งแสดงผลหน้าเมนูหลัก
------------------------------------------
mainFrm.mainloop()